In [1]:
# !pip install google-cloud-bigquery
# !pip install pyarrow

from google.cloud import bigquery
import pandas as pd
import os
import glob
import time

SERVICE_ACCOUNT_JSON = r"D:\DataAnalytics\projects\sales-store-383520-cd6fdfbcbcb6.json"
client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)


In [2]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path + '/files/', "*.csv"))

dict_dfs = {}

for f in csv_files:
    # get the file name from file and remove 'bd_' and '.csv' to use the name in the dataframe
    file_name = f.split("\\")[-1].replace('.csv', '')

    # create dinamicaly a dataframe with the name of the file
    # use ; as separator
    # change the encoding type to accept files with portuguese (brazilian) words
    globals()[f"df_{file_name}"] = pd.read_csv(f, sep=';', encoding='iso-8859-1')

    dict_dfs[f"df_{file_name}"] = globals()[f"df_{file_name}"]
    print(f"df_{file_name}")

print(len(dict_dfs))

df_address_customers
df_business_goal
df_business_unit
df_channel
df_customer_type
df_products
df_sales
df_stores
8


In [3]:
df_channel.columns = ['id_channel', 'channel']

df_address_customers.columns = ['id_address_sale', 'customer_state']

df_customer_type.columns = ['id_customer_type', 'customer_type'] 

df_stores.columns = ['id_store', 'store_code', 'start_date', 'branch', 'district', 'city', 'state']

df_business_goal.columns = ['date', 'id_store', 'id_business_unit', 'id_channel', 'sales_goal']

df_products.columns = ['id_product', 'supplier', 'product_name', 'category', 'sub-category']

df_business_unit.columns = ['id_business_unit', 'business_unit']

df_sales.columns = ['date', 'id_store', 'id_business_unit', 'id_channel', 
                        'id_product', 'id_coupon', 'id_customer', 'id_address_sale', 
                        'id_customer_type', 'items', 'gross_revenue', 'tax_value', 'costs']

In [4]:
for k, v in dict_dfs.items():

    schema=[]

    print(k)

    table_id = "sales-store-383520.raw" + k.replace('df_', '.')

    try:
        client.get_table(table_id)  # Make an API request.
        print("Table {} already exists.".format(table_id))
    except:
        table = bigquery.Table(table_id, schema=schema)
        table = client.create_table(table)  # Make an API request.
        print(
            "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
        )
    
        job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        )

        job = client.load_table_from_dataframe(v, table_id, job_config=job_config)

        while job.state != 'DONE':
            time.sleep(2)
            job.reload()
            print(job.state)

        table = client.get_table(table_id)
        print(
            "Loaded {} rows and {} columns to {}".format(
                table.num_rows, len(table.schema), table_id
            )
        )

df_address_customers
Created table sales-store-383520.raw.address_customers
RUNNING
DONE
Loaded 162774 rows and 2 columns to sales-store-383520.raw.address_customers
df_business_goal
Created table sales-store-383520.raw.business_goal
DONE
Loaded 14506 rows and 5 columns to sales-store-383520.raw.business_goal
df_business_unit
Created table sales-store-383520.raw.business_unit
DONE
Loaded 2 rows and 2 columns to sales-store-383520.raw.business_unit
df_channel
Created table sales-store-383520.raw.channel
DONE
Loaded 8 rows and 2 columns to sales-store-383520.raw.channel
df_customer_type
Created table sales-store-383520.raw.customer_type
DONE
Loaded 9 rows and 2 columns to sales-store-383520.raw.customer_type
df_products
Created table sales-store-383520.raw.products
DONE
Loaded 15259 rows and 5 columns to sales-store-383520.raw.products
df_sales
Created table sales-store-383520.raw.sales
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
DONE
Loaded 4081404 rows and 1